<a href="https://colab.research.google.com/github/Remi-Branco/Public/blob/master/Multithreading_decorator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##The multithreading decorator

This article can be found on my website at this address https://remibranco.com/?p=178

In [ ]:
from functools import wraps
import os
import concurrent.futures 

def asynchronised(*args,**kwargs):
    """
    lets you multithread any function by passing it a list you want to multithread.
    """
    
    verbose = False
    n_workers = None
    variable_to_asynchronise = None 

    for k,v in enumerate(kwargs):
        if v == "n_workers":
            n_workers = kwargs[v]
        elif v == "verbose":
            verbose = kwargs[v]
        else :
            variable_to_asynchronise = kwargs[v]

    if variable_to_asynchronise == None: #if the variable was passed as an arg
        variable_to_asynchronise =  args[0]

    if n_workers == None:
        n_workers = len(variable_to_asynchronise)

    if verbose :
        print(f" Using {n_workers} workers")
        print(kwargs)
        print(variable_to_asynchronise)
    
    def decorator(function_to_decorate):

        @wraps(function_to_decorate)
        def wrapper(*args,**kwargs):
            result = []
            with concurrent.futures.ThreadPoolExecutor(max_workers=n_workers) as e:
                futures = {e.submit(function_to_decorate,var,*args,**kwargs): var for var in variable_to_asynchronise}
                for future in concurrent.futures.as_completed(futures):
                    result.append(future.result())
            return result    
        return wrapper
    return decorator

In [ ]:
import random
import time

In [ ]:
%%time

# a list of random integers 
to_async = [random.randint(1,4) for i in range(40)]


@asynchronised(to_async)
def waiting_function(t,parameter1,parameter2):
    """
    function that sleeps for t seconds and returns t
    """ 
    ##do stuff here
    string = parameter1 + ", " + str(t) + ", " + parameter2 #some operation with parameter1 & 2
    time.sleep(t)
    ##do more stuff
    return t

results = waiting_function(parameter1="something", parameter2 = "something else" )
print(results)

[1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4, 4, 4]
CPU times: user 27.5 ms, sys: 13.5 ms, total: 40.9 ms
Wall time: 4.02 s


more parameters can be passed:
*  verbose : display the number of workers and parameters passed to the function
* n_workers : maximum number of workers (i.e. threads), by default the number of elements in the list passed to the decorator



In [ ]:
%%time

# another list of random integers
another_list = [random.randint(1,4) for i in range(20)]

print("Printed by verbose:")

@asynchronised(another_list, n_workers=8, verbose = True)  #<------ we set the maximum number of workers to 8
def waiting_function(t):
    time.sleep(t)
    return t


results = waiting_function()
print("\nresults : \n",results)

Printed by verbose:
 Using 8 workers
{'n_workers': 8, 'verbose': True}
[4, 3, 3, 1, 4, 4, 1, 3, 4, 1, 2, 3, 3, 1, 2, 1, 1, 2, 2, 4]

results : 
 [1, 1, 1, 3, 3, 3, 4, 4, 2, 4, 1, 4, 1, 1, 2, 3, 3, 2, 2, 4]
CPU times: user 52.9 ms, sys: 5.55 ms, total: 58.4 ms
Wall time: 9.01 s


The variable to multithread can be named in the decorator to improve the code readability.  In the example below, we could replace "variable_to_multithread" by "list_of_things _to_multithread".

In [ ]:
%%time

# another list of random integers 
yet_another_list = [random.randint(1,4) for i in range(20)]

@asynchronised(variable_to_multithread = yet_another_list, verbose=True)
def waiting_function(t):
    time.sleep(t)
    return t

waiting_function()


print("\nSame as above but changing the name of the 'yet_another_list' in the decorator to something that will improve code readability.\n")

@asynchronised(WHATEVER_MAKES_MORE_SENSE = yet_another_list, verbose=True)  ##<----
def waiting_function(t):
    time.sleep(t)
    return t


waiting_function()


 Using 20 workers
{'variable_to_multithread': [3, 2, 3, 1, 2, 2, 2, 4, 4, 3, 1, 4, 4, 4, 4, 1, 3, 2, 3, 2], 'verbose': True}
[3, 2, 3, 1, 2, 2, 2, 4, 4, 3, 1, 4, 4, 4, 4, 1, 3, 2, 3, 2]

Same as above but changing the name of the 'yet_another_list' in the decorator to something that will improve code readability.

 Using 20 workers
{'WHATEVER_MAKES_MORE_SENSE': [3, 2, 3, 1, 2, 2, 2, 4, 4, 3, 1, 4, 4, 4, 4, 1, 3, 2, 3, 2], 'verbose': True}
[3, 2, 3, 1, 2, 2, 2, 4, 4, 3, 1, 4, 4, 4, 4, 1, 3, 2, 3, 2]
CPU times: user 56.5 ms, sys: 11.4 ms, total: 67.9 ms
Wall time: 8.02 s
